# Cube Generation

Adapted slightly from the script by Hector Ibarra ('Cube_gen_tools')

https://github.com/sdss/lvm_early_science/tree/main


In [1]:
from astropy.table import Table
import pandas as pd
import time
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy import units as u
from scipy.ndimage import gaussian_filter1d
import sys
#from progress.bar import Bar
from tqdm.notebook import tqdm
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy import units as u
from astropy.wcs.utils import skycoord_to_pixel
from astropy.wcs.utils import pixel_to_skycoord
from astropy.wcs import WCS
import warnings
warnings.filterwarnings("ignore")
import os
from multiprocessing import Pool
from multiprocessing import cpu_count
import os.path as ptt

In [2]:
def median_a(x,lw=5,lower=10000,wave=[]):
    if len(wave) > 0:
        index=np.where(wave < lower)[0]
        index2=np.where(wave >= lower)[0]
        x1=np.copy(x)
        x=x[index]
    x_n=np.zeros(len(x))
    for i in range(0, len(x)):
        if i <= lw:
            x_d=x[0:lw]
            #x_d=reject_outliers(x_d)
            x_n[i]=np.nanmean(x_d)
        if i >= len(x)-lw:
            x_d=x[len(x)-lw:len(x)]
            #x_d=reject_outliers(x_d)
            x_n[i]=np.nanmean(x_d)
        if i > lw and i < len(x)-lw:
            x_d=x[i-lw:i+lw]
            #x_d=reject_outliers(x_d)
            x_n[i]=np.nanmean(x_d) 
    if len(wave) > 0:
        x1[index]=x_n
        x1[index2]=x_n[-1]
        x_n=x1
    return x_n

In [3]:
def read_explist(fname='Orion'):
    ft=open(fname,'r')
    tileid=[]
    tilegp=[]
    mjd=[]
    expn=[]
    for line in ft:
        if not '#' in line:
            data=line.replace('\n','').replace(' ','').split(',')
            data=list(filter(None,data))
            tileid.extend([data[0]])
            if data[0] == '11111':
                tg='0011XX'
            else:
                tg=data[0][0:4]+'XX'
            tilegp.extend([tg])
            mjd.extend([data[1]])
            expn.extend([data[2]])
    return expn,mjd,tileid,tilegp

In [4]:
def read_spall(object_name='Orion',redux_dir='',version='1.0.2.dev0'):
    from pandas import HDFStore
    try:
        hdu_list = fits.open('LVM.fits')
    except:
        print('copy and save as fits the dither summary of https://lvm-viki.lco.cl/progress.html to create the LVM.fits file') 
        return
    table_hdu = hdu_list[1]
    table_data = table_hdu.data
    target=table_data.field('target')
    tile_id=table_data.field('tile_id')
    jd=table_data.field('jd')
    mjd=np.copy(jd)
    for i in range(0, len(jd)):
        mjd[i]=jd[i]-2400000
    store0 =HDFStore(redux_dir+'/'+version+'/drpall-'+version+'.h5')
    #store0['summary'].to_csv('outputFileForTable2.csv')
    dset=store0['summary']
    mjd_t=np.array(dset['mjd'])
    tile_g=np.array(dset['tilegrp'])
    tile_idt=np.array(dset['tileid'])
    exp_n=np.array(dset['expnum'])
    store0.close()
    idt=np.zeros(len(exp_n))
    for i in range(0, len(tile_idt)):
        for j in range(0, len(tile_id)):
            if tile_idt[i] == tile_id[j]:
                if target[j] == object_name:
                    idt[i]=1
    nt=np.where(idt == 1)
    expF=list()
    expT=exp_n[nt]
    mjdT=mjd_t[nt]
    tileT=tile_idt[nt]
    tilegF=list(tile_g[nt])
    expF=[]
    mjdF=[]
    tileF=[]
    for i in range(0, len(expT)):
        expF.extend(["{:0>8}".format(expT[i])])
        mjdF.extend([str(mjdT[i])])
        tileF.extend([str(tileT[i])])
    return expF,mjdF,tileF,tilegF
            
    

In [5]:
def get_apertures(file):
    ra=[]
    dec=[]
    rad=[]
    colr=[]
    namet=[]
    l1=[]
    l2=[]
    th=[]
    typ=[]
    f=open(file,'r')
    ct=1
    for line in f:
        if not 'Region' in line and not 'fk5' in line and not 'global' in line:
            if 'circle' in line:
                data=line.replace('\n','').replace('circle(','').replace('") # color=',' , ').replace(' width=',' , ').replace(' text={',' , ').replace('}',' ')
                data=data.split(',')
                data=list(filter(None,data))
                #print(data)
                ra.extend([data[0]])
                dec.extend([data[1]])
                rad.extend([float(data[2])])
                colr.extend([data[3].replace(' ','')])
                try:
                    namet.extend([data[5].replace(' ','')])
                except:
                    namet.extend([str(int(ct))])
                l1.extend([np.nan])
                l2.extend([np.nan])
                th.extend([np.nan])    
                typ.extend(['circle'])
            if 'box' in line:
                data=line.replace('\n','').replace('box(','').replace(') # color=',' , ').replace(' width=',' , ').replace(' text={',' , ').replace('}',' ')
                data=data.split(',')
                data=list(filter(None,data))
                ra.extend([data[0]])
                dec.extend([data[1]])
                l1.extend([float(data[2].replace('"',''))])
                l2.extend([float(data[3].replace('"',''))])
                th.extend([float(data[4])])
                colr.extend([data[5].replace(' ','')])
                try:
                    namet.extend([data[7].replace(' ','')])
                except:
                    namet.extend([str(int(ct))])
                rad.extend([np.nan])    
                typ.extend(['box'])
            ct=ct+1
    ra=np.array(ra)
    dec=np.array(dec)
    rad=np.array(rad)
    colr=np.array(colr)
    namet=np.array(namet)
    typ=np.array(typ)
    l1=np.array(l1)
    l2=np.array(l2)
    th=np.array(th)
    return ra,dec,rad,l1,l2,th,colr,namet,typ

In [6]:
def extract_spec(spec,hdr,ra='',dec='',rad=1.5,pix=0.35,avgra=False):
    sky1=SkyCoord(ra+' '+dec,frame=FK5, unit=(u.hourangle,u.deg))
    val1=sky1.ra.deg
    val2=sky1.dec.deg
    wcs = WCS(hdr)
    wcs=wcs.celestial
    ypos,xpos=skycoord_to_pixel(sky1,wcs)
    print(xpos,ypos,'POS Pixel')
    val1=sky1.to_string('hmsdms')
    print(val1,'RA1,DEC1')
        
    nz,nx,ny=spec.shape
    radis=np.zeros([nx,ny])
    for i in range(0, nx):
        for j in range(0, ny):
            x_n=i-xpos
            y_n=j-ypos
            r_n=np.sqrt((y_n)**2.0+(x_n)**2.0)*pix
            radis[i,j]=r_n
    single_T=np.zeros(nz)
    nt=np.where(radis <= rad)
    for i in range(0, nz):
        tmp=spec[i,:,:]
        tmp[np.where(tmp <= 0)]=np.nan
        if avgra:
            single_T[i]=np.nanmean(tmp[nt])
        else:
            single_T[i]=np.nansum(tmp[nt])
        
    crpix=hdr["CRPIX3"]
    try:
        cdelt=hdr["CD3_3"]
    except:
        cdelt=hdr["CDELT3"]
    crval=hdr["CRVAL3"]
    wave_f=(crval+cdelt*(np.arange(nz)+1-crpix))*1e10
    
    return wave_f,single_T,xpos,ypos

In [7]:
def sycall(comand):
    import os
    linp=comand
    os.system(comand)
    
def task_wrapper(args):
    return ifu_const(*args)
    
def ifu_const(spec_ifu,specE_ifu,x_ifu_V,y_ifu_V,fibA,pix_s,sigm_s,alph_s,yo,xi,xf,nw,nl,npros,nproc,erroF):
#def ifu_const(spec_ifu0,specE_ifu0,x_ifu_V0,y_ifu_V0,fibA,pix_s,sigm_s,alph_s,yo,xi,xf,nw,nl,it,npros,nproc,wcs,ra0,dec0):    
    val=int(nl/nproc)
    a1=val*npros
    if npros < nproc-1:
        a2=val*(npros+1)
    else:
        a2=nl
    spec_fint=[]
    specE_fint=[]
    if sigm_s > fibA*3.5*2:
        radiT=sigm_s/2.0
    else:
        radiT=fibA*3.5*2/2.0
    if len(x_ifu_V.shape) > 1:
        for j in range(a1, a2):
            #skycor = pixel_to_skycoord(it,j,wcs)
            #xat=-(skycor.ra.value*3600.0-ra0)
            #yat=skycor.dec.value*3600.0-dec0
            #Rsp=np.sqrt((x_ifu_V0-(xf+xi)/2.0)**2.0)
            ##Rsp=np.sqrt((x_ifu_V0-xat)**2.0)
            #ntp=np.where(Rsp <= (fibA*3.5*2/2.0))[0]
            #spec_ifu,specE_ifu,x_ifu_V,y_ifu_V=spec_ifu0[ntp,:],specE_ifu0[ntp,:],x_ifu_V0[ntp,:],y_ifu_V0[ntp,:]
            yi=yo+pix_s*j
            yf=yo+pix_s*(j+1)
            spt_new=np.zeros(nw)
            if erroF:
                sptE_new=np.zeros(nw)
            Wgt=np.zeros(nw)
            Rspt=np.sqrt((y_ifu_V[:,0]-(yf+yi)/2.0)**2.0)
            #Rspt=np.sqrt((y_ifu_V[:,0]-yat)**2.0)
            ntpt=np.where(Rspt <= (radiT))[0]
            x_ifu_Vt=x_ifu_V[ntpt,:]
            y_ifu_Vt=y_ifu_V[ntpt,:]
            spec_ifut=spec_ifu[ntpt,:]
            if erroF:
                specE_ifut=specE_ifu[ntpt,:]
            for k in range(0, len(x_ifu_Vt[:,0])):
                Rsp=np.sqrt((x_ifu_Vt[k,:]-(xf+xi)/2.0)**2.0+(y_ifu_Vt[k,:]-(yf+yi)/2.0)**2.0)
                #Rsp=np.sqrt((x_ifu_Vt[k,:]-xat)**2.0+(y_ifu_Vt[k,:]-yat)**2.0)
                ntp=np.where((Rsp <= (radiT)) & np.isfinite(spec_ifut[k,:]) & (spec_ifut[k,:] > 0))
                Wg=np.zeros(nw)
                if len(ntp[0]) > 0:   
                    Wg[ntp]=np.exp(-(Rsp[ntp]/sigm_s)**alph_s/2.0)
                    spt_new[ntp]=spec_ifut[k,ntp]*Wg[ntp]+spt_new[ntp]
                    if erroF:
                        sptE_new[ntp]=(specE_ifut[k,ntp]**2.0)*Wg[ntp]+sptE_new[ntp]
                Wgt=Wgt+Wg
            ntp=np.where(Wgt == 0)
            if len(ntp[0]) > 0:
                Wgt[ntp]=1
            spec_fint.extend([spt_new/Wgt])
            if erroF:
                specE_fint.extend([np.sqrt(sptE_new/Wgt)])
    else:
        for j in range(a1, a2):
            #skycor = pixel_to_skycoord(it,j,wcs)
            #xat=-(skycor.ra.value*3600.0-ra0)
            #yat=skycor.dec.value*3600.0-dec0
            #Rsp=np.sqrt((x_ifu_V0-(xf+xi)/2.0)**2.0)
            ##Rsp=np.sqrt((x_ifu_V0-xat)**2.0)
            #ntp=np.where(Rsp <= (fibA*3.5*2/2.0))[0]
            #spec_ifu,specE_ifu,x_ifu_V,y_ifu_V=spec_ifu0[ntp],specE_ifu0[ntp],x_ifu_V0[ntp],y_ifu_V0[ntp]
            yi=yo+pix_s*j
            yf=yo+pix_s*(j+1)
            spax_new=0
            spaxE_new=0
            Wgt1=0
            Wgt2=0
            for k in range(0, len(x_ifu_V)):
                Rsp=np.sqrt((x_ifu_V[k]-(xf+xi)/2.0)**2.0+(y_ifu_V[k]-(yf+yi)/2.0)**2.0)
                #Rsp=np.sqrt((x_ifu_V[k]-xat)**2.0+(y_ifu_V[k]-yat)**2.0)
                if Rsp <= (radiT):   
                    Wg=np.exp(-(Rsp/sigm_s)**alph_s/2.0)
                    if np.isfinite(spec_ifu[k]): 
                        spax_new=spec_ifu[k]*Wg+spax_new
                        Wgt1=Wgt1+Wg
                    if np.isfinite(specE_ifu[k]):     
                        spaxE_new=(specE_ifu[k]**2.0)*Wg+spaxE_new
                        Wgt2=Wgt2+Wg
            if Wgt1 == 0:
                Wgt1=1
            if Wgt2 == 0:
                Wgt2=1    
            spec_fint.extend([spax_new/Wgt1])
            specE_fint.extend([np.sqrt(spaxE_new/Wgt2)])
    return ([spec_fint,specE_fint])
    
def rotate(xx,yy,angle):
    # rotate x and y cartesian coordinates by angle (in degrees)
    # about the point (0,0)
    theta = -1.*angle * np.pi / 180. # in radians
    xx1 = np.cos(theta) * xx - np.sin(theta) * yy
    yy1 = np.sin(theta) * xx + np.cos(theta) * yy
    return xx1, yy1

def make_radec(xx0,yy0,ra,dec,pa):
    xx, yy = rotate(xx0,yy0,pa)
    ra_fib = ra*3600.0 + xx/np.cos(dec*np.pi/180.) 
    dec_fib = dec*3600.0 - yy 
    return ra_fib, dec_fib    
    
def map_ifu(expnumL,nameF=None,use_slitmap=True,errors=True,pbars=True,flu16=False,multiT=False,spec_range=(None,None),fac_size=1.0,pix_s=18.5,sigm_s=18.5,alph_s=2.0,out_path='',agcam_dir='',redux_ver='1.0.2.dev0',redux_dir='',tilelist=['11111'],tileglist=['0011XX'],mjd=['0000'],scp=112.36748321030637,basename='lvmCFrame-NAME.fits',path_lvmcore=''):
    try:
        nlt=len(expnumL)
    except:
        nlt=1
    if pbars:    
        pbar=tqdm(total=nlt)     
    for i in range(0, nlt):
        if nlt == 1:
            expnum=expnumL[i]
        else:
            expnum=expnumL[i]
        expn=str(int(expnum)).zfill(8)
#        file=redux_dir+'/'+redux_ver+'/'+tileglist[i % len(tileglist)]+'/'+tilelist[i % len(tilelist)]+'/'+mjd[i % len(mjd)]+'/'+basename.replace('NAME',expn)
        file=redux_dir+'/'+basename.replace('NAME',expn)
        hdr1=fits.getheader(file,0)
        [rss, hdr0]=fits.getdata(file,'FLUX', header=True)
        if errors:
            [ivar_rss, hdre]=fits.getdata(file,'IVAR', header=True)
            erss=1/np.sqrt(ivar_rss)
        
        crpix=hdr0["CRPIX1"]
        cdelt=hdr0["CDELT1"]
        crval=hdr0["CRVAL1"]
        expT=float(hdr1['EXPTIME'])
        
        hdu_list = fits.open(file)
        table_hdu = hdu_list['SLITMAP']
        table_data = table_hdu.data
        xp=table_data.field('xpmm')*scp
        yp=table_data.field('ypmm')*scp
        Std_id=table_data.field('fiberid')-1
        ra_fib=table_data.field('ra')*3600.0
        dec_fib=table_data.field('dec')*3600.0
        typ=table_data.field('targettype')
        nt=np.where(typ == 'science')
        xp=xp[nt]
        yp=yp[nt]
        ra_fib=ra_fib[nt]
        dec_fib=dec_fib[nt]
        Std_id=Std_id[nt]
        
        
        if use_slitmap == False:
            agcam_coadd = agcam_dir+'/'+mjd[i % len(mjd)]+'/coadds/'+'lvm.sci.coadd_s'+expnum+'.fits'
            if os.path.isfile(agcam_coadd):
                agcam_hdu = fits.open(agcam_coadd)
                agcam_hdr = agcam_hdu[1].header
                w = WCS(agcam_hdr)
                cen = w.pixel_to_world(2500,1000)
                rac = cen.ra.deg  #agcam_hdr['RAMEAS']
                dec = cen.dec.deg #agcam_hdr['DECMEAS']hdr0
                PA = agcam_hdr['PAMEAS'] - 180.
                agcam_hdu.close()
            else:
                rac=hdr1["POSCIRA"]
                dec=hdr1["POSCIDE"]
                PA=hdr1["POSCIPA"]
        if i == 0:
            if use_slitmap == False:
                rac0=rac
                dec0=dec
            nx0,ny0=rss.shape
            wave0=crval+cdelt*(np.arange(ny0)+1-crpix)
            crval0=crval
            wave_1,wave_2=spec_range
            if wave_1 and wave_2:#wave_1 > np.nanmin(wave0) and
                if wave_1 < np.nanmax(wave0) and wave_2 > wave_1:# and wave_2 < np.nanmax(wave0):
                    nt=np.where((wave0 >= wave_1) & (wave0 <= wave_2))[0]
                    wave0=wave0[nt]
                    rss=rss[:,nt]
                    if errors:
                        erss=erss[:,nt]
                    crval0=np.nanmin(wave0)
                    ny0=len(wave0)
                else:
                    print('The wave Range is not well defined')  
                    return
            nfib0=len(Std_id)  
            rss_f=np.zeros([nfib0*nlt,ny0])
            rss_f[0:nfib0,:]=rss[Std_id,:]
            if errors:
                rss_ef=np.zeros([nfib0*nlt,ny0])
                rss_ef[0:nfib0,:]=erss[Std_id,:]
            x_ifu_V=np.zeros([nfib0*nlt,ny0])
            y_ifu_V=np.zeros([nfib0*nlt,ny0])
            for k in range(0, ny0):
                if use_slitmap == False:
                    ra_fib, dec_fib=make_radec(xp,yp,rac,dec,PA)
                x_ifu_V[0:nfib0,k]=ra_fib
                y_ifu_V[0:nfib0,k]=dec_fib
        else:
            nx,ny=rss.shape  
            wave=crval+cdelt*(np.arange(ny)+1-crpix)
            for j in range(0, nfib0):
                rss_f[nfib0*i+j,:]=interp1d(wave,rss[Std_id[j],:],kind='linear',bounds_error=False)(wave0)
                if errors:
                    rss_ef[nfib0*i+j,:]=interp1d(wave,erss[Std_id[j],:],kind='linear',bounds_error=False)(wave0)
            for k in range(0, ny0):
                if use_slitmap == False:
                    ra_fib, dec_fib=make_radec(xp,yp,rac,dec,PA)
                x_ifu_V[nfib0*i:nfib0*(i+1),k]=ra_fib
                y_ifu_V[nfib0*i:nfib0*(i+1),k]=dec_fib  
        if pbars:        
            pbar.update(1)     
    if pbars:
        pbar.close()      
    yot=(np.amax(y_ifu_V[:,0])+np.amin(y_ifu_V[:,0]))/2.0
    xot=(np.amax(x_ifu_V[:,0])+np.amin(x_ifu_V[:,0]))/2.0
    x_ifu_V=x_ifu_V-xot
    y_ifu_V=y_ifu_V-yot
    nw=len(wave0)
    ns=len(x_ifu_V[:,0])
    fibA=35.3
    thet=0.0
    
    nlx=int(round((np.amax([np.amax(x_ifu_V[:,0]),-np.amin(x_ifu_V[:,0])])+1)*2/pix_s))
    nly=int(round((np.amax([np.amax(y_ifu_V[:,0]),-np.amin(y_ifu_V[:,0])])+1)*2/pix_s))
    nlx=int(nlx*fac_size)
    nly=int(nly*fac_size)

    wt = WCS(naxis=2)
    wt.wcs.crpix = [nlx/2+1, nly/2+0]
    wt.wcs.cdelt = np.array([-np.cos(thet*np.pi/180.0)*pix_s/3600.0*np.cos(yot/3600.0*np.pi/180.), np.cos(thet*np.pi/180.0)*pix_s/3600.0])
    wt.wcs.crval = [xot/3600.0,yot/3600.0]
    wt.wcs.ctype = ["RA---TAN", "DEC--TAN"]

    ifu=np.zeros([nw,nly,nlx])
    ifu_e=np.ones([nw,nly,nlx])
    ifu_1=np.ones([nw,nly,nlx])
    ifu_m=np.zeros([nw,nly,nlx])
    xo=-nlx/2*pix_s
    yo=-nly/2*pix_s
    xi=xo
    xf=xo
    facto=(pix_s)**2.0/(np.pi*(fibA/2.0)**2.0)
    spec_ifu=rss_f*facto
    if errors:
        specE_ifu=rss_ef*facto 
    from multiprocessing.pool import ThreadPool
    if pbars:
        pbar=tqdm(total=nlx)
    int_spect=np.zeros(nw)
    for i in range(0, nlx):
        xi=xf
        xf=xf+pix_s
        Rsp=np.sqrt((x_ifu_V[:,0]-(xf+xi)/2.0)**2.0)
        if sigm_s > fibA*3.5*2:
            ntp=np.where(Rsp <= (sigm_s/2.0))[0]
        else:    
            ntp=np.where(Rsp <= (fibA*3.5*2/2.0))[0]
        if multiT:
            nproc=3#3#cpu_count()
            with ThreadPool(nproc) as pool:
                if errors:
                    args=[(spec_ifu[ntp,:],specE_ifu[ntp,:],x_ifu_V[ntp,:],y_ifu_V[ntp,:],fibA,pix_s,sigm_s,alph_s,yo,xi,xf,nw,nly,npros,nproc,errors) for npros in range(0, nproc)]                    
                else:
                    args=[(spec_ifu[ntp,:],None,x_ifu_V[ntp,:],y_ifu_V[ntp,:],fibA,pix_s,sigm_s,alph_s,yo,xi,xf,nw,nly,npros,nproc,errors) for npros in range(0, nproc)]
                result_l = pool.map(task_wrapper, args)
        else:
            nproc=1
            npros=0
            result_l=[]
            if errors:
                args=(spec_ifu[ntp,:],specE_ifu[ntp,:],x_ifu_V[ntp,:],y_ifu_V[ntp,:],fibA,pix_s,sigm_s,alph_s,yo,xi,xf,nw,nly,npros,nproc,errors)
            else:
                args=(spec_ifu[ntp,:],None,x_ifu_V[ntp,:],y_ifu_V[ntp,:],fibA,pix_s,sigm_s,alph_s,yo,xi,xf,nw,nly,npros,nproc,errors)
            result_l.extend([task_wrapper(args)])
        for npros in range(0, nproc):
            result=result_l[npros]
            val=int(nly/nproc)
            a1=val*npros
            if npros < nproc-1:
                a2=val*(npros+1)
            else:
                a2=nly
            ct=0
            for j in range(a1, a2):
                ifu[:,j,nlx-(i+1)]=result[0][ct]
                if errors:
                    ifu_e[:,j,nlx-(i+1)]=result[1][ct]
                ct=ct+1
        
        if pbars:
            pbar.update(1)
    if pbars:
        pbar.close()

    if ptt.exists(out_path) == False:
        os.system('mkdir -p '+out_path)
    
    if flu16:
        ifu=ifu/1e-16
        ifu_e=ifu_e/1e-16#*100
    
    h1=fits.PrimaryHDU(ifu)
    h2=fits.ImageHDU(ifu_e)
    h3=fits.ImageHDU(ifu_1)
    h4=fits.ImageHDU(ifu_m)
    head_list=[h1,h2,h3,h4]

    dx=0#+300.0/16.0/pix_s
    dy=0#+300.0/16.0/pix_s
    
    h=h1.header
    keys=list(hdr0.keys())
    for i in range(0, len(keys)):
        if not "COMMENT" in  keys[i] and not 'HISTORY' in keys[i]:
            h[keys[i]]=hdr0[keys[i]]
            h.comments[keys[i]]=hdr0.comments[keys[i]]
    h["NAXIS"]=3
    h["NAXIS3"]=nw 
    h["NAXIS1"]=nlx
    h["NAXIS2"]=nly
    #h["NDITER"]=(len(files),'Number of dither observations')
    #h["BUNIT"]= ('1E-16 erg/s/cm^2','Unit of pixel value ' )
    #h["OBJECT"]=hdr_0[0]['OBJECT']
    #h["CTYPE"] = ("RA---TAN", "DEC--TAN")
    h["CRVAL1"]=xot/3600.0
    h["CD1_1"]=-np.cos(thet*np.pi/180.)*pix_s/3600.0*np.cos(yot/3600.0*np.pi/180.)
    h["CD1_2"]=-np.sin(thet*np.pi/180.)*pix_s/3600.0*np.cos(yot/3600.0*np.pi/180.)
    h["CRPIX1"]=nlx/2+0.5+dx
    h["CTYPE1"]='RA---TAN'
    h["CRVAL2"]=yot/3600.0
    h["CD2_1"]=-np.sin(thet*np.pi/180.)*pix_s/3600.
    h["CD2_2"]=np.cos(thet*np.pi/180.)*pix_s/3600.
    h["CRPIX2"]=nly/2+0.5+dy
    h["CTYPE2"]='DEC--TAN'
    h["CUNIT1"]='deg     '                                           
    h["CUNIT2"]='deg     '
    h["CDELT3"]=cdelt
    h["CRPIX3"]=crpix
    h["CRVAL3"]=crval0
    h["CUNIT3"]=('Angstrom','Units of coordinate increment and value    ')    
    h["CTYPE3"]=('WAVE    ','Air wavelength (linear) ')
    h["RADESYS"]='FK5     '
    h["OBJSYS"]='ICRS    '
    h["EQUINOX"]=2000.00
    h["IFUCON"]=(str(int(ns))+' ','NFibers')
    if flu16:
        h["BUNIT"]='10^-16 erg/s/cm^2'
    else:
        h["BUNIT"]='erg/s/cm^2'
    hlist=fits.HDUList(head_list)
    hlist.update_extend()
    basenameC='lvmCube-NAME.fits'
    if nameF:
        file=out_path+basenameC.replace('NAME',nameF)
    else:
        file=out_path+basenameC.replace('NAME',expn)
    out_fit=file
    hlist.writeto(out_fit,overwrite=True)
    sycall('gzip -f '+out_fit)

## Genarate a data cube
Definition of main keywords:

**out_path** is the output directory for the cube

**redux_ver** is the DRP reduction version of the lvmCFrame 

**redux_dir** is the output directory of the DRP

**sigm_s** is the somth kernel size used to reconstruct the cube

**pix_s** is the spaxel size of the cube

**alph_s** is the kernel "shape" factor, a value equal of 2 returns a Gaussian kerner, larger than 2 returns a sharper kernel

**flu16** is a keyword to return a cube with units of 10^{-16} erg/s/cm2/A, if False, the units are erg/s/cm2/A

**read_spall** is an optional funtion to read the DRP drpall file, and looks for all exposures, tiles, and mjd that corresponds for an specific target

**nameF** is the output root name of the data cube

**tileglist** is a list of the tilegrouops were the lvmCFrame are located

**tilelist** is a list of the tile directories were the lvmCFrame are located

**explist** is a list of the expousure numbers of the lvmCFrame files

**mjd** is a list of the mjd directories were the lvmCFrame are located

**use_slitmap** use the astrometry saved in the slitmap header

**pbars** activate or deactivate the progress bar

**spec_range** optional, spectral range for the output cube

**read_explist** is an optional funtion to read a headerless csv file that contains the tile, mjd and exposure (in that order on the file) for a desired target

In [8]:
path_sas=os.getcwd()

# these parameters control the image reconstruction/interpolation
sigm_s=18.5#/2.0
pix_s=18.5#/2.0
alph_s=2.0


flu16=True
use_slitmap=True
nameF='test'
redux_version=''

explist=['00007332','00005752']

map_ifu(explist,nameF=nameF,flu16=flu16,multiT=True,use_slitmap=use_slitmap,sigm_s=sigm_s,pix_s=pix_s,alph_s=alph_s,agcam_dir=path_sas,redux_dir=path_sas,redux_ver=redux_version,basename='lvmSFrame-NAME.fits')



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

OSError: 265555014 requested and 245481472 written